In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import cv2
import pickle
from keras.models import load_model

In [2]:
facedetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
threshold = 0.90
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4,720)
font = cv2.FONT_HERSHEY_COMPLEX
model = load_model('MyTrainingModel.h5')

In [3]:
def preprocessing(img):
    img = img.astype('uint8')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img)
    img = img/255
    return img

def get_className(classNo):
    if classNo==0:
        return 'Why are you looking so ugly even wearing a Mask'
    elif classNo==1:
        return "Wear a Mask you Fucking Asshole"
    
while True:
    sucess, imgOriginal = cap.read()
    faces = facedetect.detectMultiScale(imgOriginal,1.3,5)
    for x,y,w,h in faces:
        crop_image = imgOriginal[y:y+h,x:x+h]
        img = cv2.resize(crop_image, (32,32))
        img = preprocessing(img)
        img = img.reshape(1,32,32,1)
        
        prediction = model.predict(img)
        classIndex = np.argmax(model.predict(img),axis=-1)
        probabilityValue=np.amax(prediction)
        if probabilityValue>threshold:
            if classIndex==0:
                cv2.rectangle(imgOriginal,(x,y),(x+w,y+h),(0,255,0),2)
                cv2.rectangle(imgOriginal,(x,y-40),(x+w,y),(0,255,0),-2)
                cv2.putText(imgOriginal, str(get_className(classIndex)),(x,y-10),font,0.75,(255,255,255),1,cv2.LINE_AA)
                
            elif classIndex==1:
                cv2.rectangle(imgOriginal,(x,y),(x+w,y+h),(50,50,255),2)
                cv2.rectangle(imgOriginal,(x,y-40),(x+w,y),(50,50,255),-2)
                cv2.putText(imgOriginal, str(get_className(classIndex)),(x,y-10),font,0.75,(255,255,255),1,cv2.LINE_AA)

        
    cv2.imshow("result", imgOriginal)
    k = cv2.waitKey(1)
    if k==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
   